# Venue Effects EDA

**Goal:** Determine whether venue/park effects are meaningful for the batted ball
outcome model. Currently `venue_id` contributes <1% feature importance.
This notebook investigates whether that's correct or whether venue effects
are real but poorly captured by a categorical ID.

**Sections:**
1. Data loading & overview
2. Outcome distributions by venue
3. Park factor calculation
4. EV/LA-controlled venue effects (residual analysis)
5. Stadium dimensions vs. outcomes
6. Spray angle effectiveness by park
7. Model residual analysis
8. Recommendations

In [ ]:
import sys
import os
from pathlib import Path

# Ensure repo root is on the path (works in both notebook and script contexts)
repo_root = Path(os.getcwd())
if (repo_root / 'Simulator').exists():
    pass  # Already at repo root
elif (repo_root.parent / 'Simulator').exists():
    repo_root = repo_root.parent  # Running from Research/
else:
    raise FileNotFoundError('Cannot find repo root. Run this from the repo root or Research/ directory.')

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

IMAGE_DIR = repo_root / 'Research' / 'venue_effects_images'
os.makedirs(IMAGE_DIR, exist_ok=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as sp_stats

from Model.data_loader import load_batted_balls_with_venue, load_games
from Model.feature_engineering import (
    calculate_spray_angle, adjust_spray_for_handedness,
    categorize_launch_angle, calculate_distance_proxy,
    is_barrel, categorize_spray_direction,
    create_features_for_prediction, HOME_PLATE_X, HOME_PLATE_Y,
)
from Simulator.constants import (
    STADIUM_DIMENSIONS, DEFAULT_STADIUM_DIMENSIONS,
    VENUE_NAME_TO_ID,
)

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

## 1. Data Loading & Overview

In [ ]:
df = load_batted_balls_with_venue()

# Filter to the 30 standard MLB venues (exclude Tokyo Dome, London, etc.)
mlb_venues = set(STADIUM_DIMENSIONS.keys())
pre_filter = len(df)
df = df[df['venue_name'].isin(mlb_venues)].copy()
print(f'Filtered to {len(mlb_venues)} MLB venues: {pre_filter:,} -> {len(df):,} events ({pre_filter - len(df)} removed)')

print(f'\nSeasons: {sorted(df["season"].unique())}')
print(f'\nVenue counts:')
print(df['venue_name'].value_counts().to_string())

In [ ]:
# Derived features
df['launch_angle_category'] = df['hitData_launchAngle'].apply(categorize_launch_angle)

df['spray_angle'] = df.apply(
    lambda r: calculate_spray_angle(r['hitData_coordinates_coordX'], r['hitData_coordinates_coordY'])
    if pd.notna(r['hitData_coordinates_coordX']) and pd.notna(r['hitData_coordinates_coordY'])
    else np.nan, axis=1
)

df['spray_angle_adj'] = df.apply(
    lambda r: adjust_spray_for_handedness(r['spray_angle'], r['batSide_code'])
    if pd.notna(r['spray_angle']) else np.nan, axis=1
)

df['spray_direction'] = df['spray_angle_adj'].apply(
    lambda x: categorize_spray_direction(x) if pd.notna(x) else 'unknown'
)

df['is_hit'] = df['eventType'].isin(['single', 'double', 'triple', 'home_run']).astype(int)
df['is_hr'] = (df['eventType'] == 'home_run').astype(int)
df['is_xbh'] = df['eventType'].isin(['double', 'triple', 'home_run']).astype(int)

bases_map = {'field_out': 0, 'double_play': 0, 'force_out': 0,
             'grounded_into_double_play': 0, 'fielders_choice': 0,
             'fielders_choice_out': 0, 'sac_fly': 0, 'sac_bunt': 0,
             'single': 1, 'double': 2, 'triple': 3, 'home_run': 4}
df['bases'] = df['eventType'].map(bases_map).fillna(0)

df['is_barrel'] = df.apply(
    lambda r: is_barrel(r['hitData_launchSpeed'], r['hitData_launchAngle']), axis=1
)

print(f'Outcome distribution:')
print(df['eventType'].value_counts().to_string())

## 2. Outcome Distributions by Venue

If venue effects are real, we should see different HR rates, hit rates, and XBH rates across parks — even in aggregate.

In [ ]:
venue_stats = df.groupby('venue_name').agg(
    n_events=('eventType', 'size'),
    hr_rate=('is_hr', 'mean'),
    hit_rate=('is_hit', 'mean'),
    xbh_rate=('is_xbh', 'mean'),
    avg_bases=('bases', 'mean'),
    avg_ev=('hitData_launchSpeed', 'mean'),
    avg_la=('hitData_launchAngle', 'mean'),
).sort_values('hr_rate', ascending=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

axes[0].barh(venue_stats.index, venue_stats['hr_rate'], color='#DC143C', alpha=0.8)
axes[0].axvline(df['is_hr'].mean(), color='black', linestyle='--', linewidth=1.5, label='League avg')
axes[0].set_xlabel('HR Rate')
axes[0].set_title('Home Run Rate by Venue', fontweight='bold')
axes[0].legend()

venue_stats_sorted = venue_stats.sort_values('hit_rate')
axes[1].barh(venue_stats_sorted.index, venue_stats_sorted['hit_rate'], color='#FFA500', alpha=0.8)
axes[1].axvline(df['is_hit'].mean(), color='black', linestyle='--', linewidth=1.5, label='League avg')
axes[1].set_xlabel('Hit Rate (BABIP)')
axes[1].set_title('Hit Rate by Venue', fontweight='bold')
axes[1].legend()

venue_stats_sorted = venue_stats.sort_values('avg_bases')
axes[2].barh(venue_stats_sorted.index, venue_stats_sorted['avg_bases'], color='#4169E1', alpha=0.8)
axes[2].axvline(df['bases'].mean(), color='black', linestyle='--', linewidth=1.5, label='League avg')
axes[2].set_xlabel('Avg Bases per BIP')
axes[2].set_title('Average Bases by Venue', fontweight='bold')
axes[2].legend()

plt.suptitle('Raw Outcome Distributions by Venue (2024-2025)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(IMAGE_DIR / '01_outcome_distributions_by_venue.png', dpi=150, bbox_inches='tight')
plt.show()

**Caveat:** Raw rates confound venue effects with team quality. A park could appear hitter-friendly simply because the home team has good hitters. Sections 4-6 control for this.

## 3. Park Factor Calculation

Classic park factors compare outcomes at a venue to outcomes in all other venues for the **same teams**. Since we don't have team-level splits in the batted ball data, we approximate by comparing each venue's outcome rate to the league average, then normalizing.

In [ ]:
league_hr_rate = df['is_hr'].mean()
league_hit_rate = df['is_hit'].mean()
league_avg_bases = df['bases'].mean()

park_factors = venue_stats[['n_events']].copy()
park_factors['HR_PF'] = (venue_stats['hr_rate'] / league_hr_rate * 100).round(0).astype(int)
park_factors['Hit_PF'] = (venue_stats['hit_rate'] / league_hit_rate * 100).round(0).astype(int)
park_factors['Bases_PF'] = (venue_stats['avg_bases'] / league_avg_bases * 100).round(0).astype(int)

park_factors = park_factors.sort_values('HR_PF', ascending=False)

print('Park Factors (100 = league average):\n')
print(park_factors[['HR_PF', 'Hit_PF', 'Bases_PF']].to_string())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pf_sorted = park_factors.sort_values('HR_PF')
colors = ['#DC143C' if v > 100 else '#4169E1' for v in pf_sorted['HR_PF']]
ax.barh(pf_sorted.index, pf_sorted['HR_PF'] - 100, color=colors, alpha=0.8)
ax.axvline(0, color='black', linewidth=1)
ax.set_xlabel('HR Park Factor (deviation from 100)')
ax.set_title('Home Run Park Factors (2024-2025)', fontweight='bold', fontsize=14)

for i, (venue, row) in enumerate(pf_sorted.iterrows()):
    ax.text(row['HR_PF'] - 100 + (1 if row['HR_PF'] >= 100 else -1), i,
            str(row['HR_PF']), va='center',
            ha='left' if row['HR_PF'] >= 100 else 'right',
            fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(IMAGE_DIR / '02_hr_park_factors.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. EV/LA-Controlled Venue Effects

The key question: **For the same exit velocity and launch angle, do outcomes differ by park?** This isolates true venue effects from "teams that play there hit harder."

We focus on "fence zone" fly balls (EV 90-110, LA 20-40) — the batted balls where park dimensions should matter most.

In [ ]:
fence_zone = df[
    (df['hitData_launchSpeed'] >= 90) &
    (df['hitData_launchSpeed'] <= 110) &
    (df['hitData_launchAngle'] >= 20) &
    (df['hitData_launchAngle'] <= 40)
].copy()

print(f'Fence-zone fly balls: {len(fence_zone):,} ({len(fence_zone)/len(df)*100:.1f}% of all BIP)')

fence_hr = fence_zone.groupby('venue_name').agg(
    n=('is_hr', 'size'),
    hr_rate=('is_hr', 'mean'),
).query('n >= 100').sort_values('hr_rate', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
colors = ['#DC143C' if v > fence_zone['is_hr'].mean() else '#4169E1'
          for v in fence_hr['hr_rate']]
ax.barh(fence_hr.index, fence_hr['hr_rate'], color=colors, alpha=0.8)
ax.axvline(fence_zone['is_hr'].mean(), color='black', linestyle='--',
           linewidth=1.5, label=f"League avg: {fence_zone['is_hr'].mean():.3f}")
ax.set_xlabel('HR Rate')
ax.set_title('HR Rate in "Fence Zone" (EV 90-110, LA 20-40\u00b0)\nControlled for Quality of Contact',
             fontweight='bold', fontsize=13)
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig(IMAGE_DIR / '03_fence_zone_hr_rate.png', dpi=150, bbox_inches='tight')
plt.show()

This is the most important chart. If venues cluster tightly around the league average, park effects are small. If there's meaningful spread (especially from known parks like Coors, Yankee Stadium), the model is missing signal.

In [ ]:
fence_hr_std = fence_hr['hr_rate'].std()
fence_hr_range = fence_hr['hr_rate'].max() - fence_hr['hr_rate'].min()

print(f'Fence-zone HR rate spread:')
print(f'  Std dev: {fence_hr_std:.4f}')
print(f'  Range:   {fence_hr_range:.4f}')
print(f'  Min:     {fence_hr["hr_rate"].min():.4f} ({fence_hr["hr_rate"].idxmin()})')
print(f'  Max:     {fence_hr["hr_rate"].max():.4f} ({fence_hr["hr_rate"].idxmax()})')

# Chi-squared test: is venue independent of HR outcome in the fence zone?
contingency = pd.crosstab(fence_zone['venue_name'], fence_zone['is_hr'])
chi2, chi2_p, dof, expected = sp_stats.chi2_contingency(contingency)
print(f'\nChi-squared test (venue vs HR in fence zone):')
print(f'  chi2 = {chi2:.2f}, p = {chi2_p:.6f}, dof = {dof}')
print(f'  Statistically significant: {"Yes" if chi2_p < 0.05 else "No"}')

## 5. Stadium Dimensions vs. Outcomes

Do physical park dimensions correlate with outcome rates? We compare CF depth, RF distance, and LF distance against HR rates and XBH rates.

In [ ]:
# Build dimensions DataFrame
dim_records = []
for venue, dims in STADIUM_DIMENSIONS.items():
    dim_records.append({
        'venue_name': venue,
        'LF': dims.get('LF'),
        'CF': dims.get('CF'),
        'RF': dims.get('RF'),
        'LCF': dims.get('LCF'),
        'RCF': dims.get('RCF'),
        'avg_depth': np.mean([dims.get('LF', 0), dims.get('CF', 0), dims.get('RF', 0)]),
    })

dims_df = pd.DataFrame(dim_records)

# Merge with venue stats
dim_analysis = dims_df.merge(
    venue_stats[['hr_rate', 'hit_rate', 'xbh_rate', 'avg_bases']].reset_index(),
    on='venue_name', how='inner'
)

# Also merge fence-zone HR rate
fence_hr_reset = fence_hr[['hr_rate']].rename(columns={'hr_rate': 'fence_hr_rate'}).reset_index()
dim_analysis = dim_analysis.merge(fence_hr_reset, on='venue_name', how='left')

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

scatter_configs = [
    ('CF', 'hr_rate', 'CF Depth vs HR Rate'),
    ('RF', 'hr_rate', 'RF Distance vs HR Rate'),
    ('LF', 'hr_rate', 'LF Distance vs HR Rate'),
    ('CF', 'fence_hr_rate', 'CF Depth vs Fence-Zone HR Rate'),
    ('avg_depth', 'hr_rate', 'Avg Depth vs HR Rate'),
    ('avg_depth', 'xbh_rate', 'Avg Depth vs XBH Rate'),
]

for ax, (x_col, y_col, title) in zip(axes.flat, scatter_configs):
    data = dim_analysis.dropna(subset=[x_col, y_col])
    ax.scatter(data[x_col], data[y_col], s=60, alpha=0.7, edgecolors='black', linewidth=0.5)

    for _, row in data.iterrows():
        if row[y_col] > data[y_col].quantile(0.85) or row[y_col] < data[y_col].quantile(0.15):
            ax.annotate(row['venue_name'].split()[-1], (row[x_col], row[y_col]),
                        fontsize=7, alpha=0.7, ha='center', va='bottom',
                        textcoords='offset points', xytext=(0, 5))

    r, p = sp_stats.pearsonr(data[x_col], data[y_col])
    ax.set_title(f'{title}\nr={r:.3f}, p={p:.3f}', fontweight='bold', fontsize=10)
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)

    z = np.polyfit(data[x_col], data[y_col], 1)
    x_line = np.linspace(data[x_col].min(), data[x_col].max(), 100)
    ax.plot(x_line, np.polyval(z, x_line), 'r--', alpha=0.5, linewidth=1.5)

plt.suptitle('Stadium Dimensions vs. Outcome Rates', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(IMAGE_DIR / '04_stadium_dimensions_vs_outcomes.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Spray Angle Effectiveness by Park

Pulled fly balls to short porches should be HRs more often. Test: does the RF distance affect pulled-fly-ball HR rate for right-handed batters (and LF distance for left-handed)?

In [ ]:
pulled_flies = df[
    (df['spray_direction'] == 'pull') &
    (df['launch_angle_category'] == 'fly_ball') &
    (df['hitData_launchSpeed'] >= 85)
].copy()

print(f'Pulled fly balls (EV >= 85): {len(pulled_flies):,}')

savefig_names = {'R': '05a_rhb_pulled_flyball_hr_vs_rf.png', 'L': '05b_lhb_pulled_flyball_hr_vs_lf.png'}
for side, fence_col, side_label in [('R', 'RF', 'RHB pulled to RF'), ('L', 'LF', 'LHB pulled to LF')]:
    subset = pulled_flies[pulled_flies['batSide_code'] == side]

    venue_hr = subset.groupby('venue_name').agg(
        n=('is_hr', 'size'),
        hr_rate=('is_hr', 'mean'),
    ).query('n >= 20')

    venue_hr = venue_hr.merge(dims_df[['venue_name', fence_col]], on='venue_name')

    r, p = sp_stats.pearsonr(venue_hr[fence_col], venue_hr['hr_rate'])

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(venue_hr[fence_col], venue_hr['hr_rate'], s=60, alpha=0.7,
               edgecolors='black', linewidth=0.5)

    for _, row in venue_hr.iterrows():
        if row['hr_rate'] > venue_hr['hr_rate'].quantile(0.85) or row['hr_rate'] < venue_hr['hr_rate'].quantile(0.15):
            ax.annotate(row['venue_name'].split()[-1], (row[fence_col], row['hr_rate']),
                        fontsize=8, ha='center', va='bottom',
                        textcoords='offset points', xytext=(0, 5))

    z = np.polyfit(venue_hr[fence_col], venue_hr['hr_rate'], 1)
    x_line = np.linspace(venue_hr[fence_col].min(), venue_hr[fence_col].max(), 100)
    ax.plot(x_line, np.polyval(z, x_line), 'r--', alpha=0.5, linewidth=1.5)

    ax.set_xlabel(f'{fence_col} Distance (ft)')
    ax.set_ylabel('HR Rate on Pulled Fly Balls')
    ax.set_title(f'{side_label}: Pulled Fly Ball HR Rate vs {fence_col} Distance\n'
                 f'r={r:.3f}, p={p:.3f}', fontweight='bold')
    plt.tight_layout()
    plt.savefig(IMAGE_DIR / savefig_names[side], dpi=150, bbox_inches='tight')
    plt.show()

## 7. Model Residual Analysis

Run the current model on all batted balls and check if prediction errors cluster by venue. If the model captures venue effects well, residuals should be uniform across parks. If not, systematic under/over-prediction by venue signals missing information.

In [ ]:
import joblib

pipeline = joblib.load(str(repo_root / 'Model' / 'batted_ball_model.pkl'))

# Only rows with complete data
pred_df = df.dropna(subset=['hitData_coordinates_coordX', 'hitData_coordinates_coordY',
                            'hitData_launchSpeed', 'hitData_launchAngle',
                            'batSide_code', 'venue_name']).copy()

pred_df['venue_id'] = pred_df['venue_name'].map(VENUE_NAME_TO_ID).fillna('22')

print(f'Predicting on {len(pred_df):,} batted balls with complete data...')

In [ ]:
from Model.feature_engineering import (
    calculate_distance_proxy, is_barrel,
    PULL_THRESHOLD, OPPO_THRESHOLD,
)

# Build features in bulk
pred_df['spray_angle_adj'] = pred_df.apply(
    lambda r: adjust_spray_for_handedness(
        calculate_spray_angle(r['hitData_coordinates_coordX'], r['hitData_coordinates_coordY']),
        r['batSide_code']
    ), axis=1
)
pred_df['spray_angle_abs'] = pred_df['spray_angle_adj'].abs()
pred_df['distance_proxy'] = pred_df.apply(
    lambda r: calculate_distance_proxy(r['hitData_launchSpeed'], r['hitData_launchAngle']), axis=1
)
pred_df['is_barrel_feat'] = pred_df.apply(
    lambda r: is_barrel(r['hitData_launchSpeed'], r['hitData_launchAngle']), axis=1
)
pred_df['is_pulled'] = (pred_df['spray_angle_adj'] < PULL_THRESHOLD).astype(int)
pred_df['is_opposite'] = (pred_df['spray_angle_adj'] > OPPO_THRESHOLD).astype(int)
pred_df['pulled_hard'] = ((pred_df['is_pulled'] == 1) & (pred_df['hitData_launchSpeed'] >= 95)).astype(int)
pred_df['oppo_hard'] = ((pred_df['is_opposite'] == 1) & (pred_df['hitData_launchSpeed'] >= 95)).astype(int)
pred_df['spray_ev_interaction'] = pred_df['spray_angle_adj'] * pred_df['hitData_launchSpeed']

la_cat = pred_df['hitData_launchAngle'].apply(categorize_launch_angle)
pred_df['pulled_ground_ball'] = ((pred_df['is_pulled'] == 1) & (la_cat == 'ground_ball')).astype(int)
pred_df['oppo_line_drive'] = ((pred_df['is_opposite'] == 1) & (la_cat == 'line_drive')).astype(int)

# Assemble features in model's expected order
feature_cols = [
    'hitData_launchSpeed', 'hitData_launchAngle', 'distance_proxy',
    'spray_angle_adj', 'spray_angle_abs', 'is_barrel_feat',
    'is_pulled', 'is_opposite', 'pulled_hard', 'oppo_hard',
    'spray_ev_interaction', 'pulled_ground_ball', 'oppo_line_drive',
]
cat_cols = ['launch_angle_category', 'spray_direction', 'venue_id']

X = pred_df[feature_cols + cat_cols].copy()
X = X.rename(columns={'is_barrel_feat': 'is_barrel'})

probs = pipeline.predict_proba(X)
pred_df['pred_bases'] = probs[:, 1]*1 + probs[:, 2]*2 + probs[:, 3]*3 + probs[:, 4]*4
pred_df['pred_hr_prob'] = probs[:, 4]
pred_df['pred_hit_prob'] = probs[:, 1] + probs[:, 2] + probs[:, 3] + probs[:, 4]

pred_df['residual_bases'] = pred_df['bases'] - pred_df['pred_bases']
pred_df['residual_hr'] = pred_df['is_hr'] - pred_df['pred_hr_prob']

print('Predictions complete.')

In [ ]:
residuals_by_venue = pred_df.groupby('venue_name').agg(
    n=('residual_bases', 'size'),
    mean_resid_bases=('residual_bases', 'mean'),
    mean_resid_hr=('residual_hr', 'mean'),
    std_resid_bases=('residual_bases', 'std'),
).query('n >= 500').sort_values('mean_resid_hr')

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

colors = ['#DC143C' if v > 0 else '#4169E1' for v in residuals_by_venue['mean_resid_hr']]
axes[0].barh(residuals_by_venue.index, residuals_by_venue['mean_resid_hr'], color=colors, alpha=0.8)
axes[0].axvline(0, color='black', linewidth=1)
axes[0].set_xlabel('Mean HR Residual (actual - predicted)')
axes[0].set_title('HR Prediction Residuals by Venue\n(+) = model underpredicts HRs',
                   fontweight='bold', fontsize=12)

resid_sorted = residuals_by_venue.sort_values('mean_resid_bases')
colors = ['#DC143C' if v > 0 else '#4169E1' for v in resid_sorted['mean_resid_bases']]
axes[1].barh(resid_sorted.index, resid_sorted['mean_resid_bases'], color=colors, alpha=0.8)
axes[1].axvline(0, color='black', linewidth=1)
axes[1].set_xlabel('Mean Bases Residual (actual - predicted)')
axes[1].set_title('Bases Prediction Residuals by Venue\n(+) = model underpredicts total bases',
                   fontweight='bold', fontsize=12)

plt.suptitle('Model Residual Analysis by Venue', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(IMAGE_DIR / '06_model_residuals_by_venue.png', dpi=150, bbox_inches='tight')
plt.show()

If residuals are systematically positive for some parks (e.g., Coors) and negative for others, the model is not fully capturing venue effects.

In [ ]:
# ANOVA: are residuals different across venues?
groups = [group['residual_bases'].values for _, group in pred_df.groupby('venue_name')
          if len(group) >= 500]
f_stat, anova_p = sp_stats.f_oneway(*groups)
print(f'One-way ANOVA on bases residuals across venues:')
print(f'  F = {f_stat:.2f}, p = {anova_p:.6f}')
print(f'  Significant venue effects in residuals: {"Yes" if anova_p < 0.05 else "No"}')

# Effect size (eta-squared)
ss_between = sum(len(g) * (g.mean() - pred_df['residual_bases'].mean())**2 for g in groups)
ss_total = ((pred_df['residual_bases'] - pred_df['residual_bases'].mean())**2).sum()
eta_sq = ss_between / ss_total
print(f'  Effect size (eta-squared): {eta_sq:.6f}')
print(f'  Interpretation: {"Negligible" if eta_sq < 0.01 else "Small" if eta_sq < 0.06 else "Medium" if eta_sq < 0.14 else "Large"}')

## 8. Summary & Recommendations

In [ ]:
print('=' * 70)
print('VENUE EFFECTS SUMMARY')
print('=' * 70)
print(f'\n1. Raw HR rate range across venues:  {venue_stats["hr_rate"].min():.4f} - {venue_stats["hr_rate"].max():.4f}')
print(f'   (league avg: {league_hr_rate:.4f})')
print(f'\n2. Fence-zone HR rate range:          {fence_hr["hr_rate"].min():.4f} - {fence_hr["hr_rate"].max():.4f}')
print(f'   (controlling for EV 90-110, LA 20-40)')
print(f'\n3. Chi-squared test (venue vs fence-zone HR): p = {chi2_p:.6f}')
print(f'\n4. Model residual ANOVA: F = {f_stat:.2f}, p = {anova_p:.6f}, eta-sq = {eta_sq:.6f}')
print(f'\n5. Top 3 parks where model underpredicts HRs:')
top_under = residuals_by_venue.nlargest(3, 'mean_resid_hr')
for venue, row in top_under.iterrows():
    print(f'   {venue}: +{row["mean_resid_hr"]:.4f} mean HR residual (n={int(row["n"]):,})')
print(f'\n6. Top 3 parks where model overpredicts HRs:')
top_over = residuals_by_venue.nsmallest(3, 'mean_resid_hr')
for venue, row in top_over.iterrows():
    print(f'   {venue}: {row["mean_resid_hr"]:.4f} mean HR residual (n={int(row["n"]):,})')

### Recommendations

Based on the analysis above, consider these approaches:

**If venue effects are statistically significant but small (eta-sq < 0.01):**
- The current categorical `venue_id` approach is reasonable
- The <1% feature importance may accurately reflect reality
- No model changes needed

**If venue effects are significant with meaningful effect size (eta-sq >= 0.01):**
- Replace categorical `venue_id` with continuous park features:
  - `cf_depth`: Center field distance in feet
  - `rf_depth`: Right field distance
  - `lf_depth`: Left field distance
  - `avg_fence_depth`: Average of LF/CF/RF
- These give the model more signal than 30 unrelated categories
- Consider adding elevation data (Coors at 5,280 ft is the obvious case)

**If spray angle x park interactions are significant:**
- Add interaction features: `spray_angle_adj x fence_depth_in_spray_direction`
- This directly encodes "is this fly ball headed toward a short porch?"